In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.txt')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),
                                                 df['has_covid'],
                                                 test_size=0.2)
X_train

,age,gender,fever,cough,city
71,75,Female,104.0,Strong,Delhi
75,5,Male,102.0,Mild,Kolkata
65,69,Female,102.0,Mild,Bangalore
82,24,Male,98.0,Mild,Kolkata
90,59,Female,99.0,Strong,Delhi
...,...,...,...,...,...
66,51,Male,104.0,Mild,Kolkata
8,19,Female,100.0,Strong,Bangalore
85,16,Female,103.0,Mild,Bangalore
18,64,Female,98.0,Mild,Bangalore


In [8]:
#without column tranformer

In [15]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']]) #use double parenthesis
#it gives dataframe while single one gives series, and sklearn classes need
#input as 2d ie dataframe

# also the test data
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [16]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [17]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [18]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [19]:
X_train_transformed = np.concatenate((X_train_age,
                                      X_train_fever,
                                      X_train_gender_city,
                                      X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,
                                     X_test_fever,
                                     X_test_gender_city,
                                     X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [20]:
#with column tranformer

In [21]:
from sklearn.compose import ColumnTransformer

In [28]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')
#remainder is for what to do with rmaining coumns- passthrough or drop

In [30]:
X_train_transformed=transformer.fit_transform(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [31]:
X_train_transformed.shape

(80, 7)

In [33]:
X_test_transformed=transformer.transform(X_test)

In [35]:
X_test_transformed.shape

(20, 7)